In [2]:
import torch
import torch.nn as nn

In [3]:
torch.manual_seed(1)
rnn_layer = nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True)

Definimos un módulo recurrente, con una única capa con 2 neuronas, que se conecta con 5 neuronas (features) de entrada.
Veamos como quedan sus parámetros.

In [4]:
for param, value in rnn_layer.named_parameters():
    print(f"{param}: {value}")

weight_ih_l0: Parameter containing:
tensor([[ 0.3643, -0.3121, -0.1371,  0.3319, -0.6657],
        [ 0.4241, -0.1455,  0.3597,  0.0983, -0.0866]], requires_grad=True)
weight_hh_l0: Parameter containing:
tensor([[ 0.1961,  0.0349],
        [ 0.2583, -0.2756]], requires_grad=True)
bias_ih_l0: Parameter containing:
tensor([-0.0516, -0.0637], requires_grad=True)
bias_hh_l0: Parameter containing:
tensor([ 0.1025, -0.0028], requires_grad=True)


In [5]:
w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0
print('W_xh shape:', w_xh.shape)
print('W_hh shape:', w_hh.shape)
print('b_xh shape:', b_xh.shape)
print('b_hh shape:', b_hh.shape)

W_xh shape: torch.Size([2, 5])
W_hh shape: torch.Size([2, 2])
b_xh shape: torch.Size([2])
b_hh shape: torch.Size([2])


Vemos que tenemos:
- una matriz con los pesos que unen las 2 neuronas con los 5 inputs.
- una matriz con los pesos de los estados anteriores de las 2 neuronas con ellas mismas en el paso actual.
- los 2 sesgos de las conexiones de entrada (uno para cada neurona)
- los 2 sesgos de las conexiones recurrentes (uno para cada neurona)

Creamos una única instancia con una secuencia 3 pasos de tiempo, con los datos de 5 features (e.g. temperatura, presión, precipitación).

In [6]:
## secuencia de una instancia, con 3 pasos de tiempo, con 5 features de entrada
x_seq = torch.tensor([[1.0]*5, [2.0]*5, [3.0]*5]).float()
print(x_seq)
print(x_seq.shape)

tensor([[1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3.]])
torch.Size([3, 5])


Necesitamos crear el eje del batch que espera toda capa.

In [7]:
torch.reshape(x_seq, (1, 3, 5))

tensor([[[1., 1., 1., 1., 1.],
         [2., 2., 2., 2., 2.],
         [3., 3., 3., 3., 3.]]])

Vamos a procesar ese batch (de **1 única instancia**) a través del módulo recurrente.

In [8]:
## output of the simple RNN:
output, hn = rnn_layer(torch.reshape(x_seq, (1, 3, 5)))
print(output)
print(output.shape)
print(hn)
print(hn.shape)

tensor([[[-0.3520,  0.5253],
         [-0.6842,  0.7607],
         [-0.8649,  0.9047]]], grad_fn=<TransposeBackward1>)
torch.Size([1, 3, 2])
tensor([[[-0.8649,  0.9047]]], grad_fn=<StackBackward0>)
torch.Size([1, 1, 2])


El procesamiento del batch (de 1 única instancia) produce dos tensores:
- un **tensor de salidas**: La única capa recurrente tiene dos neuronas, la secuencia tiene un largo de 3 pasos. El tensor de salida incluirá la secuencia de los outputs progresivos de los 3 pasos. Es por eso que tiene una organización [batch=1, pasos=3, neuronas=2]
- un **tensor de estados escondidos**, con el último valor de salida de la celda, pero con el mismo rango.

Podemos hacer el proceso a pie.
Veamoslo para el primer paso de tiempo.

In [9]:
torch.reshape(x_seq[0], (1, 5))

tensor([[1., 1., 1., 1., 1.]])

In [10]:
b_xh

Parameter containing:
tensor([-0.0516, -0.0637], requires_grad=True)

In [40]:
[i.item() for i in b_xh]

[-0.051554277539253235, -0.0636589378118515]

In [41]:
[i.item() for i in b_hh]

[0.1024908497929573, -0.002824766095727682]

In [45]:
[i.detach().numpy() for i in w_hh]

[array([0.19612378, 0.03488284], dtype=float32),
 array([ 0.2582553 , -0.27556023], dtype=float32)]

In [47]:
paso=0

# input del primer paso
xt = torch.reshape(x_seq[0], (1, 5))
print("input:\n", xt)
print("matriz Wxh:\n", w_xh)
print("matriz Wxh transpuesta:\n", torch.transpose(w_xh, 0, 1))
print("sesgos bxh:\n",b_xh)
# ht combinación lineal: x*Wxh + bxh
ht = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh
print("\ncomb lineal x*Wxh + bhh:\n", ht)

input:
 tensor([[1., 1., 1., 1., 1.]])
matriz Wxh:
 Parameter containing:
tensor([[ 0.3643, -0.3121, -0.1371,  0.3319, -0.6657],
        [ 0.4241, -0.1455,  0.3597,  0.0983, -0.0866]], requires_grad=True)
matriz Wxh transpuesta:
 tensor([[ 0.3643,  0.4241],
        [-0.3121, -0.1455],
        [-0.1371,  0.3597],
        [ 0.3319,  0.0983],
        [-0.6657, -0.0866]], grad_fn=<TransposeBackward0>)
sesgos bxh:
 Parameter containing:
tensor([-0.0516, -0.0637], requires_grad=True)

comb lineal x*Wxh + bhh:
 tensor([[-0.4702,  0.5864]], grad_fn=<AddBackward0>)


In [48]:
prev_h = torch.zeros((ht.shape))
print("\ninput ht-1:\n", prev_h)
print("matriz Whh transpuesta:\n", torch.transpose(w_hh, 0, 1))
print("sesgos bhh:\n",b_hh)
# salida
ot = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
print("\nsalida combinación lineal:\n", ot)
ot = torch.tanh(ot)
print("salida activación:\n", ot)


input ht-1:
 tensor([[0., 0.]])
matriz Whh transpuesta:
 tensor([[ 0.1961,  0.2583],
        [ 0.0349, -0.2756]], grad_fn=<TransposeBackward0>)
sesgos bhh:
 Parameter containing:
tensor([ 0.1025, -0.0028], requires_grad=True)

salida combinación lineal:
 tensor([[-0.3677,  0.5836]], grad_fn=<AddBackward0>)
salida activación:
 tensor([[-0.3520,  0.5253]], grad_fn=<TanhBackward0>)


Vamos a hacerlo para la secuencia de 3 pasos:

In [29]:
## manually computing the output:
out_man = []
for t in range(3):
    xt = torch.reshape(x_seq[t], (1, 5))
    print(f'Time step {t} =>')
    print(' Input :', xt.numpy())

    ht = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh
    print('   Hidden :', ht.detach().numpy())

    if t > 0:
        prev_h = out_man[t-1]
    else:
        prev_h = torch.zeros((ht.shape))

    print('   prev :', prev_h.detach().numpy())
    # print('   w_hh :', torch.transpose(w_hh, 0, 1))
    # print('   out_man :', out_man)
    # print('   b_hh :', b_hh.detach().numpy())
    temp = torch.matmul(prev_h, torch.transpose(w_hh, 0, 1))  + b_hh
    print('   hh :', temp.detach().numpy())


    ot = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
    ot = torch.tanh(ot)
    out_man.append(ot)
    print('\n Output (manual) :', ot.detach().numpy())
    print(' RNN output :', output[:, t].detach().numpy())
    print()

Time step 0 =>
 Input : [[1. 1. 1. 1. 1.]]
   Hidden : [[-0.4701929   0.58639044]]
   prev : [[0. 0.]]
   hh : [[ 0.10249085 -0.00282477]]

 Output (manual) : [[-0.3519801   0.52525216]]
 RNN output : [[-0.3519801   0.52525216]]

Time step 1 =>
 Input : [[2. 2. 2. 2. 2.]]
   Hidden : [[-0.88883156  1.2364398 ]]
   prev : [[-0.3519801   0.52525216]]
   hh : [[ 0.05178148 -0.2384641 ]]

 Output (manual) : [[-0.68424344  0.76074266]]
 RNN output : [[-0.68424344  0.76074266]]

Time step 2 =>
 Input : [[3. 3. 3. 3. 3.]]
   Hidden : [[-1.3074702  1.8864892]]
   prev : [[-0.68424344  0.76074266]]
   hh : [[-0.0051687 -0.3891647]]

 Output (manual) : [[-0.8649416  0.9046636]]
 RNN output : [[-0.8649416  0.9046636]]

